# Test Matching Insee/ETS

## INSEE

- https://s3.console.aws.amazon.com/s3/object/calfdata/INSEE/Stock/ETS/
        - INSEE/Stock/ETS/StockEtablissement_utf8.csv
        
```
['siren', 'siret']
```

## INPI

- https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/Stock_processed/
    - INPI/TC_1/Stock_processed/initial_ETS.gz
    - INPI/TC_1/Stock_processed/initial_ETS.json
    
Colonnes test:

```
["Siren","Date_Immatriculation", "Date_Clôture", "Date_Greffe"]
```

## Sauvegarde

* La liste des SIREN matchés sera sauvegardée selon leur nature et origine
  * nature → ACTES/COMPTES/ETS/etc
  * origine → initial/partiel/new/evt

Les matchés seront sauvegardé dans calfdata/SIRETISATION/matche/ au format suivant:

* insee_nature_origine_matche.gz
    * ex: insee_pm_initial_matche.gz
    
    

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/
* Infogreffe
  * https://www.infogreffe.fr/


Le siège ne donne pas de nouveau SIRET, il indique seulement le lieu de la juridiction

In [ ]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
#import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [ ]:
#instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
#bucket = 'calfdata'

In [ ]:
# instanciate AWS connection
#AWS_connection = aws.aws_instantiate(instance_aws, bucket)

## Preparation fichiers

## Matching établissement principal

Ici, on filtre les variables communes pour l'INSEE & INPI établissements secondaires.

### Candidats

**INSEE**

https://www.sirene.fr/sirene/public/static/liste-variables

- numeroVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/numeroVoieEtablissement
- indiceRepetitionEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/indiceRepetitionEtablissement
- typeVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/typeVoieEtablissement
- libelleVoieEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleVoieEtablissement
- complementAdresseEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/complementAdresseEtablissement
- codeCommuneEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codeCommuneEtablissement
- libelleCommuneEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCommuneEtablissement
- codePostalEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codePostalEtablissement
- codeCedexEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codeCedexEtablissement
- libelleCedexEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCedexEtablissement
- distributionSpecialeEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/distributionSpecialeEtablissement
- libelleCommuneEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libelleCommuneEtrangerEtablissement
- codePaysEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/codePaysEtrangerEtablissement
- libellePaysEtrangerEtablissement: https://www.sirene.fr/sirene/public/static/liste-variables/libellePaysEtrangerEtablissement

**INPI**

- Adresse_Ligne1/Adresse_Ligne2/Adresse_Ligne3: Selon les greffes, l’adresse (n°+ voie) sera présente soit en ligne1 adresse, soit en ligne2 adresse.
Toutes les lignes d’adresse ne sont pas nécessairement renseignées.



### Créer fichier toutes les possibilités communes

In [ ]:
communes = pd.read_csv('temp_local\communes-01012019.csv').set_index('ncc').reindex(columns = ['nccenr', 'libelle'])#.unstack()
communes.loc[lambda x: x['libelle'].isin(['Châtillon-sur-Chalaronne'])]

In [ ]:
communes = (pd.read_csv('temp_local\communes-01012019.csv')
            .set_index('ncc')
            .reindex(columns=['nccenr', 'libelle'])
            .assign(
    noaccent=lambda x: x['nccenr'].str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'),
    nccenr_noponc=lambda x: x['nccenr'].str.replace('[^\w\s]', ' '),
    libelle_noponc=lambda x: x['libelle'].str.replace('[^\w\s]', ' '),
    noaccent_noponc=lambda x: x['noaccent'].str.replace('[^\w\s]', ' '),
    uppercase=lambda x: x.index,
    nccenr_uppercase=lambda x: x['nccenr'].str.upper(),
    libelle_uppercase=lambda x: x['libelle'].str.upper(),
    noaccent_uppercase=lambda x: x['noaccent'].str.upper(),
    nccenr_noponc_uppercase=lambda x: x['nccenr_noponc'].str.upper(),
    libelle_noponc_uppercase=lambda x: x['libelle_noponc'].str.upper(),
    noaccent_noponc_uppercase=lambda x: x['noaccent_noponc'].str.upper(),
    nccenr_lowercase=lambda x: x['nccenr'].str.lower(),
    libelle_lowercase=lambda x: x['libelle'].str.lower(),
    noaccent_lowercase=lambda x: x['noaccent'].str.lower(),
    nccenr_noponc_lowercase=lambda x: x['nccenr_noponc'].str.lower(),
    libelle_noponc_lowercase=lambda x: x['libelle_noponc'].str.lower(),
    noaccent_noponc_lowercase=lambda x: x['noaccent_noponc'].str.lower(),
    nccenr_noarrond1=lambda x: x['nccenr'].str.replace(
        'er Arrondissement', ''),
    uppercase_noarrond1=lambda x: x['uppercase'].str.replace(
        'ER ARRONDISSEMENT', ''),
    lowercase_noarrond1=lambda x: x['nccenr_lowercase'].str.replace(
        'er arrondissement', ''),
    nccenr_noarrond=lambda x: x['nccenr'].str.replace('e Arrondissement', ''),
    uppercase_noarrond=lambda x: x['uppercase'].str.replace(
        'E ARRONDISSEMENT', ''),
    lowercase_noarrond=lambda x: x['nccenr_lowercase'].str.replace(
        'e arrondissement', ''),
)
)

for n in communes.columns:
    var_ = '{}_ST'.format(n)
    var_1 = '{}_st'.format(n)
    var_2 = '{}_St'.format(n)
    
    communes[var_] = communes[n].str.replace('SAINT', 'ST')
    communes[var_1] = communes[n].str.replace('Saint', 'st')
    communes[var_2] = communes[n].str.replace('Saint', 'St')
    
    var_ = '{}_Sbar'.format(n)
    var_1 = '{}_sbar'.format(n)
    
    communes[var_] = communes[n].str.replace('SUR', 'S/')
    communes[var_1] = communes[n].str.replace('sur', 's/')
    
communes = (communes
            .stack()
            .rename('possibilite')
            .reset_index()
            .drop(columns='level_1')
            .drop_duplicates(subset=['possibilite']))
communes.head()

In [ ]:
#insee = AWS_connection.url_instance_bucket(path_file = 'INSEE/Stock/ETS/StockEtablissement_utf8.csv')
#ets = AWS_connection.url_instance_bucket(path_file = 'INPI/TC_1/Stock_processed/initial_ETS.gz')
#ets

In [ ]:
insee = r"\temp_local\StockEtablissement_utf8.csv"
ets = r"\temp_local\initial_ETS.gz"

In [ ]:
# load data into dataframes
data_insee_ = dd.read_csv(insee,
                          usecols=['siren',
                                   'siret',
                                   "numeroVoieEtablissement",
                                   "indiceRepetitionEtablissement",
                                   "typeVoieEtablissement",
                                   "libelleVoieEtablissement",
                                   "complementAdresseEtablissement",
                                   "codeCommuneEtablissement",
                                   "libelleCommuneEtablissement",
                                   "codePostalEtablissement",
                                   "codeCedexEtablissement",
                                   "libelleCedexEtablissement",
                                   "distributionSpecialeEtablissement",
                                   "libelleCommuneEtrangerEtablissement",
                                   "codePaysEtrangerEtablissement",
                                   "libellePaysEtrangerEtablissement",
                                   "dateCreationEtablissement"
                                   ],
                          dtype={'siren': 'object',
                                 'siret': 'object',
                                 "numeroVoieEtablissement":'object',
                                   "indiceRepetitionEtablissement":'object',
                                   "typeVoieEtablissement":'object',
                                   "libelleVoieEtablissement":'object',
                                   "complementAdresseEtablissement":'object',
                                   "codeCommuneEtablissement":'object',
                                   "libelleCommuneEtablissement":'object',
                                   "codePostalEtablissement":'object',
                                   "codeCedexEtablissement":'object',
                                   "libelleCedexEtablissement":'object',
                                   "distributionSpecialeEtablissement":'object',
                                   "libelleCommuneEtrangerEtablissement":'object',
                                   "codePaysEtrangerEtablissement":'object',
                                   "libellePaysEtrangerEtablissement":'object'
                                 }
                          )

data_ets_ = (dd.read_csv(ets,
                         usecols=[
                             'Type',
                             'Siren',
                             'Code_Postal',
                             'Code_Commune',
                             'Adresse_Ligne1',
                             'Adresse_Ligne2',
                             'Adresse_Ligne3',
                             'Ville',
                             'Pays'
                         ],
                         dtype={
                             'Type': 'object',
                             'Siren': 'object',
                             'Code_Postal': 'object',
                             'Code_Commune': 'object',
                             'Adresse_Ligne1': 'object',
                             'Adresse_Ligne2': 'object',
                             'Adresse_Ligne3': 'object',
                             'Ville':'object',
                             'Pays':'object'
                         },
                         compression='gzip',
                         blocksize=None,
                         low_memory=False
                         )
             .compute()
             .rename(columns={"Siren": "siren"})
             .loc[lambda x: ~x['Type'].isin(['SIE'])]
             )

In [ ]:
data_insee_.shape

In [ ]:
data_ets_.shape

In [ ]:
siren_inpi = data_ets_['siren'].drop_duplicates()
len(siren_inpi)

In [ ]:
len(siren_inpi)/data_ets_.shape[0]

In [ ]:
subset_insee = (data_insee_
                .loc[data_insee_['siren'].isin(siren_inpi.to_list())]
                .loc[data_insee_['dateCreationEtablissement'] <= "2018-01-01"]
                .assign(
                libelleCommuneEtablissement = lambda x:
                    x['libelleCommuneEtablissement'].str.replace('-', ' ')
                )
                .compute()
               )

Siren INPI mais pas INSEE -> Cette entreprise a exercé son droit d'opposition auprès de l'INSEE. Ses données ne peuvent pas être diffusées publiquement.

temp insee - > gagner du temps pendant la periode de dév
temp inpi - > gagner du temps pendant la periode de dév

In [ ]:
siren_to_remove = siren_inpi.loc[lambda x : ~x.isin(subset_insee['siren'])]
len(siren_to_remove)

In [ ]:
df_siren_to_find = data_ets_.loc[lambda x:
                                 (~x['siren'].isin(siren_to_remove))    
                                 ]
len(df_siren_to_find)

In [ ]:
#df_siren_to_find.to_csv('temp_inpi.csv', index = False)

### Nan variables matching 

on exclue les variables avec que des nan dans les variables candidates

-> on les traitera après

In [ ]:
import numpy as np
siren_fullna = df_siren_to_find.loc[lambda x:
                      (x['Adresse_Ligne1'].isin([np.nan]))
                     & (x['Adresse_Ligne2'].isin([np.nan]))
                     & (x['Adresse_Ligne3'].isin([np.nan]))
                     & (x['Code_Postal'].isin([np.nan]))
                     & (x['Ville'].isin([np.nan]))
                     & (x['Code_Commune'].isin([np.nan]))
                     ]['siren']

In [ ]:
df_siren_to_find = df_siren_to_find.loc[lambda x:
                                 (~x['siren'].isin(siren_fullna))
                                 ]
len(siren_fullna)

### Nombres d'ets par SIREN INSEE

On calcule le nombre d'etb pour le fichier INSEE.

In [ ]:
subset_insee_count = subset_insee.merge(
    (subset_insee
     .groupby('siren')['siren']
     .count()
     .rename('count')
     .reset_index())
)

In [ ]:
df_siren_to_find = df_siren_to_find.merge(
    (df_siren_to_find
     .groupby('siren')['siren']
     .count()
     .rename('count')
     .reset_index()
    )
)

In [ ]:
df_siren_to_find.shape[0]

Insee enlever les tirets dans la ville

In [ ]:
def siren_unique(df):
    """
    """
    print("Nombre total obs: {}".format(len(df)))
    count_ = (df
              .groupby('siren')['siren']
              .count()
              .rename('count')
              .reset_index()
              .groupby('count')['count']
              .count()
              .reset_index(name='total_count')
              .set_index('count')
              # .compute()
              .assign(pct=lambda x: x/x.sum())
              .iloc[:10, :]
              .style
              .format('{:,.2%}', subset=['pct'])
              )
    return count_

Quick stat

In [ ]:
siren_unique(df = subset_insee_count)

In [ ]:
siren_unique(df = df_siren_to_find)

## Step 0: Clean ville

Ajout matching des communes pour retrouver le libelé commune de l'INSEE

ATTENTION, il faut nétoyer la variables ville dans l'INSEE. Veuillez regarder le fichier `communes.xlsx` pour voir les différents problèmes

ex: 
- CEDEX, cedex, digit, (d+), 

attention, l'arrondissement peut être mis entre parenthèse 

- MARSEILLE (7E)

- process:
    - creer variables avec numeric seulement
    - recreer ville 2 si test pas NAN pour avoir l'arrondissement
    - virer les differentes informations dans ville via regex

In [ ]:
import numpy as np
regex = 'CEDEX|cedex|Cedex|\([^)]*\)|/\s\s+/|^\d+\s|\s\d+\s|\s\d+$|\d+|\.|\--|COMMUNE DE |COMMUNE DE|commune de |commune de|Commune de |Commune de |\s$'
test_adress = df_siren_to_find.copy()
test_adress['test'] =test_adress['Ville'].str.extract(r'(\d+)')
test_adress['Ville_clean'] = test_adress['Ville'].str.replace(regex,'')
test_adress['Ville_clean'] = test_adress['Ville_clean'].str.replace('\s$|\s^','')
test_adress['ville2'] = np.where(
    np.logical_and(
         ~test_adress['test'].isin([np.nan]),
        test_adress['test'].str.len() <=2
    )
   ,
    test_adress['Ville_clean'] + '' + test_adress['test'].astype(str),
    test_adress['Ville_clean']
)

test_adress = test_adress.merge(communes,
                         left_on='ville2',
                         right_on='possibilite',
                         how='left',
                         indicator=True)

test_adress = pd.concat([
    test_adress.loc[lambda x: x['_merge'].isin(['both'])],
    (test_adress
     .loc[lambda x: x['_merge'].isin(['left_only'])]
     .drop(columns=['ncc', 'possibilite', '_merge'])
     .merge(communes,
            left_on='Ville_clean',
            right_on='possibilite',
            how='left',
            indicator=True)
     )

])

test_adress = pd.concat([
    test_adress.loc[lambda x: x['_merge'].isin(['both'])],
    (test_adress
     .loc[lambda x: x['_merge'].isin(['left_only'])]
     .drop(columns=['ncc', 'possibilite', '_merge'])
     .assign(
         noaccent=lambda x: x['Ville_clean'].str.normalize('NFKD')
         .str.encode('ascii', errors='ignore')
         .str.decode('utf-8'))
     ).merge(communes,
             left_on='noaccent',
             right_on='possibilite',
             how='left',
             indicator=True)])
test_adress.groupby('_merge')["_merge"].count()


In [ ]:
test_adress = test_adress.drop(columns = '_merge')

In [ ]:
test_adress.shape

In [ ]:
test_adress.head()

## Process

On ne match que les SIREN dont la date de création est inférieur a 2018

1) ~Step : Calculer le nombre de `nan` dans les colonnes de matching~

2) ~Step : Compter le nombre de SIRET by SIREN~

2) Step 2:  merge sur siren et code postal

3) Step 3:  merge sur siren et code commune



### Step 1: Match uniquement les 1 dans INSEE/INPI

On enlève les matches du dataframe `df_siren_to_find` et on ajoute les `left_only`.

Pareil pour l'INSEE pour gagner en mémoire.

In [ ]:
m1_unique = (
    subset_insee_count.loc[lambda x: x['count'].isin([1])]
 .merge(test_adress.loc[lambda x: x['count'].isin([1])],
         how='left',indicator=True)
       )

In [ ]:
m1_unique.groupby('_merge')["_merge"].count()

In [ ]:
to_remove_ = m1_unique.loc[lambda x: x['_merge'].isin(['both'])]['siren'].to_list()

In [ ]:
test_adress = test_adress.loc[lambda x: ~x['siren'].isin(to_remove_)]
#subset_insee_count = subset_insee_count.loc[lambda x: ~x['siren'].isin(to_remove_)]

In [ ]:
siren_unique(df = test_adress)

In [ ]:
siren_unique(df = subset_insee_count)

Exemple de SIREN qui ont seulement une ligne dans l'INPI mais plusieurs SIRET dans l'INSEE.

- 813543063
- 800897092

In [ ]:
test_adress.loc[lambda x: x['count'] ==1].head(2)

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['813543063'])]

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['800897092'])]

In [ ]:
subset_insee_count.isna().sum().sort_values()

In [ ]:
test_adress.isna().sum().sort_values()

### Step 2: Merging 

Dans cette partie, on va merger sur plusieurs candidats. La plupart des SIREN peuvent être matché via le code postal, code commune, ou ville directement. Si un SIREN a plusieurs SIRET dans la même ville ou code postal, il fera l'objet d'une recherche plus poussée.

Trois cas de figure découle du merge:

- 1) Merge forte pertinence
- 2) merge pertinence moyenne -> plusieurs SIRET pour un même candidat
- 3) Unmerge

#### 1:  merge sur siren et Ville

- Merge sur siren & libelleCommuneEtablissement|Ville_clean

In [ ]:
def merge(df_insee, df_inpi, left_on, right_on):
    """
    """
    # match
    data_merged_1 = (df_insee
                     .merge(
                         df_inpi,
                         how='right',
                         left_on=left_on,
                         right_on=right_on,
                         indicator=True,
                         suffixes=['_insee', '_inpi'])
                     )

    # count
    count_ = (data_merged_1
              .loc[lambda x: x['_merge'].isin(['both'])]
              .groupby(['siren', 'ncc'])['siren']
              .count()
              .rename('count')
              .reset_index()
              .groupby('count')['count']
              .count()
              .reset_index(name='total_count')
              .set_index('count')
              .assign(pct=lambda x: x/x.sum())
              .iloc[:10, :]
              .style
              .format('{:,.2%}', subset=['pct'])
              )

    # detail match
    detail = data_merged_1.groupby('_merge')["_merge"].count()

    # cas de figure 2
    siren_fig2 = (data_merged_1
                  .loc[lambda x: x['_merge'].isin(['both'])]
                  .groupby(['siren', 'ncc'])['siren']
                  .count()
                  .rename('count')
                  .loc[lambda x:x > 1]
                  .reset_index('ncc')
                  .index
                  )

    # non matche
    siren_nmatched = (data_merged_1
                      .loc[lambda x: x['_merge'].isin(['right_only'])]['siren']
                      .to_list()
                      )
    new_unmatch = df_inpi.loc[lambda x: x['siren'].isin(siren_nmatched)]

    dic_ = {

        'count_': count_,
        'detail': detail,
        'siren_fig': siren_fig2,
        'size_fig2': len(siren_fig2),
        'new_unmatch': new_unmatch,
    }

    return dic_

In [ ]:
test_city = merge(df_insee =subset_insee_count,
      df_inpi =test_adress,
      left_on=['siren', 'libelleCommuneEtablissement'],
      right_on=['siren', 'ncc'])

In [ ]:
test_city['detail']

In [ ]:
test_city['size_fig2']

In [ ]:
test_city['count_']

In [ ]:
test_city['new_unmatch'].shape

Exemple de cas de figure 2: merge pertinence moyenne -> plusieurs SIRET pour un même candidat:

- 200000560

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['200000560'])]

In [ ]:
test_adress.loc[lambda x: x['siren'].isin(['200000560'])]

In [ ]:
siren_unique(df = test_city['new_unmatch'])

#### 2:  merge sur siren et code postal

In [ ]:
test_cp = merge(df_insee =subset_insee_count,
      df_inpi = test_city['new_unmatch'],
      left_on= ['siren', 'codePostalEtablissement'],
      right_on= ['siren', 'Code_Postal'])

In [ ]:
test_cp['detail']

In [ ]:
test_cp['size_fig2']

In [ ]:
test_cp['count_']

In [ ]:
siren_unique(df = test_cp['new_unmatch'])

### 3:  merge sur siren et code commune

- Merge sur siren & codeCommuneEtablissement

In [ ]:
test_com = merge(df_insee =subset_insee_count,
      df_inpi = test_cp['new_unmatch'],
      left_on= ['siren', 'codeCommuneEtablissement'],
      right_on= ['siren', 'Code_Commune'])

In [ ]:
test_com['detail']

In [ ]:
test_com['size_fig2']

In [ ]:
test_com['count_']

In [ ]:
siren_unique(df = test_com['new_unmatch'])

## A Verifier

Il reste a véfifier les cas de figure 2 et les unmatches

In [ ]:
### Total a matcher avant ville/code postal/commune
test_adress.shape[0]

In [ ]:
### total cas de figure 2 

In [ ]:
test_city['size_fig2'] + test_cp['size_fig2']+ test_com['size_fig2']

In [ ]:
### total unmatch 
test_com['new_unmatch'].shape[0]

In [ ]:
### nombres totals SIREN a matcher

test_city['size_fig2'] + test_cp['size_fig2']+ test_com['size_fig2'] + test_com['new_unmatch'].shape[0]

Verification le nombre a matcher correspond bien a 2M

In [ ]:
pd.concat(
    [
        df_siren_to_find.loc[lambda x: x['siren'].isin(test_city['siren_fig'])],
        df_siren_to_find.loc[lambda x: x['siren'].isin(test_cp['siren_fig'])],
        df_siren_to_find.loc[lambda x: x['siren'].isin(test_com['siren_fig'])],
        test_com['new_unmatch'] 
    ]

).shape

In [ ]:
test_city['size_fig2']

In [ ]:
test_city['size_fig2']

In [ ]:
(866618 / df_siren_to_find.shape[0]) * 100

In [ ]:
### Match avec ville/code postal/commune
test_adress.shape[0] - test_city['size_fig2'] + test_cp['size_fig2'] + \
test_com['size_fig2'] - test_com['new_unmatch'].shape[0]

In [ ]:
test_adress.loc[lambda x : x['siren'].isin(['200000560'])]

In [ ]:
subset_insee_count.loc[lambda x : x['siren'].isin(['200000560'])]

### Match avec adresse

On selectionne uniquement ceux pas matché.
Pour accélerer la recherche, on utilise que le sous ensemble de siren a vérifier dans le fichier INSEE

Verifier si on peut matcher avec le numéro de l'adresse -> au cas ou principal et secondaire dans le même endroit

Il faut retravailler les adresses:

- Upper case

La recherche se fait sur le libellé adresse. Dans l'INSEE, pas de numéro de voie, ni de typologie (rue, avenue, etc)

ALL: Allée
AV: Avenue
BD: Boulevard
CAR: Carrefour
CHE: Chemin
CHS: Chaussée
CITE: Cité
COR: Corniche
CRS: Cours
DOM: Domaine
DSC: Descente
ECA: Ecart
ESP: Esplanade
FG: Faubourg
GR: Grande Rue
HAM: Hameau
HLE: Halle
IMP: Impasse
LD: Lieu dit
LOT: Lotissement
MAR: Marché
MTE: Montée
PAS: Passage
PL: Place
PLN: Plaine
PLT: Plateau
PRO: Promenade
PRV: Parvis
QUA: Quartier
QUAI: Quai
RES: Résidence
RLE: Ruelle
ROC: Rocade
RPT: Rond Point
RTE: Route
RUE: Rue
SEN: Sente - Sentier
SQ: Square
TPL: Terre-plein
TRA: Traverse
VLA: Villa
VLGE: Village

In [ ]:
additional = ["Avenue",
"Boulevard",
"Carrefour",
"Chemin",
"Chaussee",
"Cite",
"Corniche",
"Cours",
"Domaine",
"Descente",
"Ecart",
"Esplanade",
"Faubourg",
"Grande Rue",
"Hameau",
"Halle",
"Impasse",
"Lieu dit",
"Lotissement",
"Marche",
"Montee",
"Passage",
"Place",
"Plaine",
"Plateau",
"Promenade",
"Parvis",
"Quartier",
"Quai",
"Residence",
"Ruelle",
"Rocade",
"Rond Point",
"Route",
"Rue",
"Sentier",
"Square",
"Terre plein",
"Traverse",
"Villa",
"Village"
'bp', 'cedex']

In [ ]:
import nltk
#nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('french')
stop_words.extend(additional)
upper_stop = [i.upper() for i in stop_words]

In [ ]:
def create_split_adress(x):
    """
    """
    split_ = x.str.split().to_list()
    
    #split_ = ''.join(str(e) for e in split_)
    #reg = '|'.join(split_)
    return  split_


def create_regex_adress(x):
    """
    """
    try:
        split_ = [i + "$" for i in x]
        reg = '|'.join(split_)
    except:
        reg = np.nan
    return  reg

import re

def find_regex(regex, test_str, siret):
    """
    """
    try:
        matches = re.search(regex, test_str)
        if matches:
            return siret
        else:
            return np.nan
    except:
        return np.nan
    
def prepare_adress(df):
    """
    """
    #temp_adresse = m3['unmerged'].loc[lambda x: ~
    #                                  x['siren'].isin(siren_count_1_found)].copy()
    #sous_ensemble_insee = subset_insee_count.loc[lambda x: x['siren'].isin(
    #    temp_adresse['siren'].to_list())]

    temp_adresse = df.assign(
        Adresse_Ligne1_clean=lambda x: x['Adresse_Ligne1'].str.normalize(
            'NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace('[^\w\s]|\d+', '')
        .str.upper(),
        Adresse_Ligne2_clean=lambda x: x['Adresse_Ligne2'].str.normalize(
            'NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace('[^\w\s]|\d+', '')
        .str.upper(),
        Adresse_Ligne3_clean=lambda x: x['Adresse_Ligne3'].str.normalize(
            'NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace('[^\w\s]|\d+', '')
        .str.upper()
    )
    temp_adresse['Adresse_Ligne1_clean'] = (temp_adresse['Adresse_Ligne1_clean']
                                            .apply(lambda x:
                                                   ' '.join([word for word in
                                                             str(x).split() if
                                                             word not in 
                                                             (upper_stop)]))
                                            )

    temp_adresse['Adresse_Ligne2_clean'] = (temp_adresse['Adresse_Ligne2_clean']
                                            .apply(lambda x:
                                                   ' '.join([word for word in
                                                             str(x).split() if
                                                             word not in 
                                                             (upper_stop)]))
                                           )
                                            

    temp_adresse['Adresse_Ligne3_clean'] = (temp_adresse['Adresse_Ligne3_clean']
                                            .apply(lambda x:
                                                   ' '.join([word for word in
                                                             str(x).split() if
                                                             word not in 
                                                             (upper_stop)]))
                                           )
                                            

    temp_adresse = temp_adresse.assign(
        Adresse_Ligne1_clean_split=lambda x:
        create_split_adress(x['Adresse_Ligne1_clean']),
        Adresse_Ligne2_clean_split=lambda x:
        create_split_adress(x['Adresse_Ligne2_clean']),
        Adresse_Ligne3_clean_split=lambda x:
        create_split_adress(x['Adresse_Ligne3_clean'])
    )

    temp_adresse['Adresse_Ligne1_clean_reg'] = temp_adresse['Adresse_Ligne1_clean_split'].apply(lambda x:
                                                                                                create_regex_adress(x))
    temp_adresse['Adresse_Ligne2_clean_reg'] = temp_adresse['Adresse_Ligne2_clean_split'].apply(lambda x:
                                                                                                create_regex_adress(x))
    temp_adresse['Adresse_Ligne3_clean_reg'] = temp_adresse['Adresse_Ligne3_clean_split'].apply(lambda x:
                                                                                                create_regex_adress(x))

    return temp_adresse

#def lookupInseeInpi(df_insee, siren, regex_):
#    """
#    """
#    try:
#        siret_ = df_insee.loc[lambda x: 
#                                  x['siren'].isin([siren])
#                      & x['libelleVoieEtablissement'].str.contains(
#                          regex_, 
#                          case = False, 
#                          regex = True)
#                      ]['siret']
#        return siret_.values[0]
#    except:
#        return np.nan 

### Test sur `test_city`

On fait le test seulement sur le sous ensemble `test_city`. Cela évite d'avoir un immense dataframe avec le merge de l'insee

On peut faire le matching sur:

- Adresse_Ligne1_clean_reg
- Adresse_Ligne2_clean_reg
- Adresse_Ligne3_clean_reg

In [ ]:
#test1 = prepare_adress(
#    df_siren_to_find.loc[lambda x: x['siren'].isin(test_city['siren_fig'])])

In [ ]:
from dask.diagnostics import ProgressBar
from dask.multiprocessing import get
import re
import dask.dataframe as dd
import pandas as pd
import numpy as np
pbar = ProgressBar()
pbar.register()

# load data into dataframes
subset_insee_count = pd.read_csv('subset_insee_count.csv',
                          usecols=['siren',
            'siret',
            'libelleCommuneEtablissement',
            'libelleVoieEtablissement',
            'numeroVoieEtablissement'
                                   ],
                          dtype={'siren': 'object',
                                 'siret': 'object',
                                 "libelleCommuneEtablissement":'object',
                                   "libelleVoieEtablissement":'object',
                                   "numeroVoieEtablissement":'object'
                                 }
                          )



def find_regex(regex, test_str, siret):
    """
    """
    try:
        matches = re.search(regex, test_str)
        if matches:
            return siret
        else:
            return np.nan
    except:
        return np.nan

In [ ]:
test1 = (pd.read_csv('test_city.csv',
                         usecols=[
                             'siren', 'Type', 'Code_Postal', 'Ville',
                     'Adresse_Ligne1_clean_reg'
                         ],
                         dtype={
                             'siren': 'object',
                             'Type': 'object',
                             'Code_Postal': 'object',
                             'Ville': 'object',
                             'Adresse_Ligne1_clean_reg': 'object',
                         },
                         chunksize=250000,
                         #low_memory=False
                         )
             )

#### test_1: `Adresse_Ligne1_clean_reg`

Pour accelerer le code, on utilise uniquement les adresses sans les na et on filtre l'insee

In [ ]:
#test_inpi_1 = test1.loc[lambda x: ~x['Adresse_Ligne1_clean_reg'].isin(['nan$'])]
#test_inpi_1.shape

In [ ]:
#test_insee= (subset_insee_count
#             .loc[lambda x: x['siren'].isin([test_inpi_1['siren']])])

In [ ]:
#subset_insee_count.shape

POur acceleter le calcul, on convertit la df en Dask et on fait un map partition

In [ ]:
test1.head()

In [ ]:
len(test1)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for chunk in tqdm(pd.read_csv('test_city.csv',
                         usecols=[
                             'siren', 'Type', 'Code_Postal', 'Ville',
                     'Adresse_Ligne1_clean_reg'
                         ],
                         dtype={
                             'siren': 'object',
                             'Type': 'object',
                             'Code_Postal': 'object',
                             'Ville': 'object',
                             'Adresse_Ligne1_clean_reg': 'object',
                         },
                         chunksize=1000000,
                         #low_memory=False
                         )):
    print('merge')
    temp = (chunk.loc[~chunk['Adresse_Ligne1_clean_reg'].isin(['nan$'])]
        .merge(
        subset_insee_count,
        on='siren')
        )
    print('apply')
    if not temp.empty:
        temp['siret_test1'] = (temp.apply(lambda x:
                 find_regex(
                     x['Adresse_Ligne1_clean_reg'],
                     x['libelleVoieEtablissement'],
                     x['siret']), axis=1)
    
    )
        temp = temp.loc[lambda x: ~x['siret_test1'].isin([np.nan])]

In [ ]:
temp.head()

In [ ]:
chunk.head()

In [ ]:
subset_insee_count.loc[lambda x: x['siren'].isin(['712980432'])]

In [ ]:
#test_insee = dd.from_pandas(
#    subset_insee_count[[
#            'siren',
#            'siret',
#            'libelleCommuneEtablissement',
#            'libelleVoieEtablissement',
#            'numeroVoieEtablissement']],
#    npartitions=30)

In [ ]:
temp = (test1.loc[~test1['Adresse_Ligne1_clean_reg'].isin(['nan$'])]
        .merge(
        subset_insee_count,
        on='siren')
        )

temp['siret_test1'] = temp.map_partitions(
        lambda df:
        df.apply(lambda x:
                 find_regex(
                     x['Adresse_Ligne1_clean_reg'],
                     x['libelleVoieEtablissement'],
                     x['siret']), axis=1)
    .dropna()
    ).compute()

In [ ]:
temp.head()

In [ ]:
def merge_adresse(df_insee, df_inpi):
    """
    """

    temp = (df_inpi
            .merge(
        df_insee,
        on='siren')
    )

    ddata['siret_test1'] = ddata.map_partitions(
        lambda df:
        df.apply(lambda x:
                 find_regex(
                     x['Adresse_Ligne1_clean_reg'],
                     x['libelleVoieEtablissement'],
                     x['siret']), axis=1)
    ).compute(scheduler='threads')
    # cas de figure 2
    fig2 = (ddata
            .compute()
            .loc[lambda x: ~x['siret_test1'].isin([np.nan])]
            .groupby(['siren', 'libelleVoieEtablissement'])['siren']
            .count()
            .rename('count')
            .loc[lambda x:x > 1]
            .reset_index('libelleVoieEtablissement')
            .index
            )

    # Unmatched
    list_unmatch = (ddata
                    .compute()
                    .loc[lambda x: x['siret_test1'].isin([np.nan])]['siren']
                    .drop_duplicates())

    # df match

    df_match = (ddata
                .compute()
                .loc[lambda x: ~x['siret_test1'].isin([np.nan])]
                .drop(columns=['siret_test1'])
                )
    
    ### perc matched
    pct_match = df_match.shape[0]/len(ddata)

    dic_ = {

        'count_': df_match.shape[0],
        'detail': pct_match,
        'siren_fig': fig2,
        'size_fig2': len(fig2),
        'new_unmatch': list_unmatch,
    }
    
    return dic_

In [ ]:
%%time
test_adresse1=  merge_adresse(df_insee= subset_insee_count,
              df_inpi= test_inpi_1)

In [ ]:
test_adresse1['size_fig2']

In [ ]:
ddata = dd.from_pandas(temp.head(1000), npartitions=30)

In [ ]:
ddata.compute()

In [ ]:

ddata['siret_test1'] = ddata.map_partitions(lambda df:
                     df.apply(lambda x:
                              find_regex(
                                  x['Adresse_Ligne1_clean_reg'],
                                  x['libelleVoieEtablissement'],
                                  x['siret']), axis=1)
                            ).compute(scheduler='threads')

In [ ]:
(ddata
 .compute()
 .loc[lambda x: ~x['siret_test1'].isin([np.nan])]
 .drop(columns = ['siret_test1'])
).shape[0]/len(ddata)

In [ ]:
len(ddata)

Subset les SIREN avec la même adresses pour faire une vérification plus poussée

In [ ]:
(ddata
 .compute()
 .loc[lambda x: ~x['siret_test1'].isin([np.nan])]
 .groupby(['siren', 'libelleVoieEtablissement'])['siren']
 .count()
 .rename('count')
 .loc[lambda x:x > 1]
 .reset_index('libelleVoieEtablissement')
 .index )

In [ ]:
((300504 * 0.620) /1000)/60

#### test_2

In [ ]:
%%time
test_2 = temp_adresse.loc[lambda x: ~x['test_2'].isin(['nan'])]
test_2['siret_2'] = test_1.apply(lambda x: lookupInseeInpi(
    siren = x['siren'],
    regex_ = x['test_2']),
    axis = 1 )

In [ ]:
test_2['siret_2'].isna().sum()

In [ ]:
test_2.shape

In [ ]:
test_2.loc[lambda x : ~x['siret_2'].isin([np.nan])].head()

In [ ]:
test_2.loc[lambda x : ~x['siret_2'].isin([np.nan])].shape

In [ ]:
(sous_ensemble_insee
 .loc[lambda x: x['siren'].isin(["394674881"])]
)

#### test_3

In [ ]:
%%time
test_3 = temp_adresse.loc[lambda x: ~x['test_3'].isin(['nan'])]
test_3['siret_3'] = test_1.apply(lambda x: lookupInseeInpi(
    siren = x['siren'],
    regex_ = x['test_3']),
    axis = 1 )

In [ ]:
test_3.loc[lambda x : ~x['siret_3'].isin([np.nan])].shape

In [ ]:
test_3.loc[lambda x : ~x['siret_3'].isin([np.nan])].head()

In [ ]:
(sous_ensemble_insee
 .loc[lambda x: x['siren'].isin(["302556832"])]
)

In [ ]:
lookupInseeInpi(siren = '302556832',
                regex_ = '^AVE$|^MICHEL$|^JOURDAN$')

temp1.head(

In [ ]:
import os
os.remove('communes.xlsx')
temp1.loc[lambda x: x['_merge'].isin(['left_only'])].drop_duplicates('ville2').to_excel('communes.xlsx')

In [ ]:
temp.assign(url = lambda x :
            'https://data.inpi.fr/entreprises/' + x["Siren"] )